In [1]:
import os
import time
import glob
import re
import pandas as pd
from functools import reduce
from klepto.archives import dir_archive

import numpy as np
from tqdm.auto import tqdm
import coffea.processor as processor
from coffea.processor.accumulator import AccumulatorABC
from coffea.analysis_objects import JaggedCandidateArray
from coffea.btag_tools import BTagScaleFactor
from coffea import hist
import pandas as pd
import uproot_methods
import uproot
import awkward
import copy

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

from Tools.config_helpers import *
from Tools.helpers import mergeArray, mt

from Tools.objects import Collections
from Tools.cutflow import Cutflow

# This just tells matplotlib not to open any
# interactive windows.
matplotlib.use('Agg')

In [2]:
def pad_and_flatten(val): 
    try:
        return val.pad(1, clip=True).fillna(0.).flatten()#.reshape(-1, 1)
    except AttributeError:
        return val.flatten()

#os.environ['KERAS_BACKEND'] = 'theano'
#from keras.models import load_model

In [3]:
import sys
sys.setrecursionlimit(10000)
print(sys.getrecursionlimit())

10000


In [10]:
class analysisProcessor(processor.ProcessorABC):
    """Processor used for running the analysis"""
    def __init__(self):
        
        ## load b-tag SFs
        #self.btag_sf = BTagScaleFactor(os.path.expandvars("$TWHOME/data/DeepCSV_102XSF_V1.btag.csv.gz", "reshape")

        ## load the NN
        #self.model = load_model('../ML/data/training.h5')
        #self.stds  = pd.read_json('../ML/data/stds.json').squeeze()
        #self.means = pd.read_json('../ML/data/means.json').squeeze()
        
        # we can use a large number of bins and rebin later
        dataset_axis        = hist.Cat("dataset",   "Primary dataset")
        pt_axis             = hist.Bin("pt",        r"$p_{T}$ (GeV)", 1000, 0, 1000)
        p_axis              = hist.Bin("p",         r"$p$ (GeV)", 1000, 0, 2500)
        ht_axis             = hist.Bin("ht",        r"$H_{T}$ (GeV)", 500, 0, 5000)
        mass_axis           = hist.Bin("mass",      r"M (GeV)", 1000, 0, 2000)
        eta_axis            = hist.Bin("eta",       r"$\eta$", 60, -5.5, 5.5)
        delta_axis          = hist.Bin("delta",     r"$\delta$", 100,0,10 )
        multiplicity_axis   = hist.Bin("multiplicity",         r"N", 20, -0.5, 19.5)
        norm_axis           = hist.Bin("norm",         r"N", 25, 0, 1)

        self._accumulator = processor.dict_accumulator({
            "MET_pt_baseline" :          hist.Hist("Counts", dataset_axis, pt_axis),
            "HT_baseline" :              hist.Hist("Counts", dataset_axis, ht_axis),
            "mtb_min_baseline" :         hist.Hist("Counts", dataset_axis, mass_axis),
            "MET_pt" :          hist.Hist("Counts", dataset_axis, pt_axis),
            "HT" :              hist.Hist("Counts", dataset_axis, ht_axis),
            "mtb_min" :         hist.Hist("Counts", dataset_axis, mass_axis),
            "MET_pt_SR" :       hist.Hist("Counts", dataset_axis, pt_axis),
            "HT_SR" :           hist.Hist("Counts", dataset_axis, ht_axis),
            "mtb_min_SR" :      hist.Hist("Counts", dataset_axis, mass_axis),
            "lead_AK8_pt" :     hist.Hist("Counts", dataset_axis, pt_axis),
            "W_pt" :            hist.Hist("Counts", dataset_axis, pt_axis),
            "H_pt" :            hist.Hist("Counts", dataset_axis, pt_axis),
            "W_eta" :           hist.Hist("Counts", dataset_axis, eta_axis),
            "H_eta" :           hist.Hist("Counts", dataset_axis, eta_axis),
            
            "N_b" :             hist.Hist("Counts", dataset_axis, multiplicity_axis),
            "N_AK4" :           hist.Hist("Counts", dataset_axis, multiplicity_axis),
            "N_AK8" :           hist.Hist("Counts", dataset_axis, multiplicity_axis),
            "N_H" :             hist.Hist("Counts", dataset_axis, multiplicity_axis),
            "N_W" :             hist.Hist("Counts", dataset_axis, multiplicity_axis),
            
            "WH_deltaPhi":      hist.Hist("Counts", dataset_axis, delta_axis),
            "WH_deltaR":        hist.Hist("Counts", dataset_axis, delta_axis),
            "bb_deltaPhi":      hist.Hist("Counts", dataset_axis, delta_axis),
            "bb_deltaR":        hist.Hist("Counts", dataset_axis, delta_axis),
            "min_dphiJetMet4":  hist.Hist("Counts", dataset_axis, delta_axis),
            "dphiDiJet":        hist.Hist("Counts", dataset_axis, delta_axis),
            "dphiDiFatJet":     hist.Hist("Counts", dataset_axis, delta_axis),
            
            'mC750_l1':         processor.defaultdict_accumulator(int),
            'WJets':            processor.defaultdict_accumulator(int),
            'QCD':              processor.defaultdict_accumulator(int),
            'TTJets':           processor.defaultdict_accumulator(int),
            'ZNuNu':            processor.defaultdict_accumulator(int),
            'ST':               processor.defaultdict_accumulator(int),
            'ttW':              processor.defaultdict_accumulator(int),
            'ttZ':              processor.defaultdict_accumulator(int),
            'WW':               processor.defaultdict_accumulator(int),
            'WZ/ZZ':            processor.defaultdict_accumulator(int),
            'LL':               processor.defaultdict_accumulator(int),
            'totalEvents':      processor.defaultdict_accumulator(int),
            'test1':            processor.defaultdict_accumulator(float),
        })

    @property
    def accumulator(self):
        return self._accumulator

    def process(self, df):
        """
        Processing function. This is where the actual analysis happens.
        """
        output = self.accumulator.identity()
        dataset = df["dataset"]
        cfg = loadConfig()
        
        ## MET -> can switch to puppi MET
        met_pt  = df["MET_pt"]
        met_phi = df["MET_phi"]
        
        ## Muons
        muon = JaggedCandidateArray.candidatesfromcounts(
            df['nMuon'],
            pt = df['Muon_pt'].content,
            eta = df['Muon_eta'].content,
            phi = df['Muon_phi'].content,
            mass = df['Muon_mass'].content,
            miniPFRelIso_all=df['Muon_miniPFRelIso_all'].content,
            looseId =df['Muon_looseId'].content
            )
        muon = muon[(muon.pt > 10) & (abs(muon.eta) < 2.4) & (muon.looseId) & (muon.miniPFRelIso_all < 0.2)]
        #muon = Collections(df, "Muon", "tightTTH").get() # this needs a fix for DASK
        
        electrons = JaggedCandidateArray.candidatesfromcounts(
            df['nElectron'],
            pt=df['Electron_pt'].content, 
            eta=df['Electron_eta'].content, 
            phi=df['Electron_phi'].content,
            mass=df['Electron_mass'].content,
            pdgid=df['Electron_pdgId'].content,
            mini_iso=df['Electron_miniPFRelIso_all'].content
        )
        
        ## Electrons
        electron = JaggedCandidateArray.candidatesfromcounts(
            df['nElectron'],
            pt = df['Electron_pt'].content,
            eta = df['Electron_eta'].content,
            phi = df['Electron_phi'].content,
            mass = df['Electron_mass'].content,
            miniPFRelIso_all=df['Electron_miniPFRelIso_all'].content,
            cutBased=df['Electron_cutBased'].content
            )
        electron = electron[(electron.pt>10) & (abs(electron.eta) < 2.4) & (electron.miniPFRelIso_all < 0.1) &  (electron.cutBased >= 1)]
        #electron = Collections(df, "Electron", "tightTTH").get() # this needs a fix for DASK
        
        tau = JaggedCandidateArray.candidatesfromcounts(
            df['nTau'],
            pt=df['Tau_pt'].content, 
            eta=df['Tau_eta'].content, 
            phi=df['Tau_phi'].content,
            mass=df['Tau_mass'].content,
            decaymode=df['Tau_idDecayMode'].content,
            newid=df['Tau_idMVAnewDM2017v2'].content,
        )
        tau = tau[(tau.pt > 20) & (abs(tau.eta) < 2.4) & (tau.decaymode) & (tau.newid >= 8)]
        
        isotrack = awkward.JaggedArray.zip(
            pt=df['IsoTrack_pt'], 
            eta=df['IsoTrack_eta'], 
            phi=df['IsoTrack_phi'], 
            rel_iso=df['IsoTrack_pfRelIso03_all'], 
        )
        isotrack = isotrack[(isotrack.pt > 10) & (abs(isotrack.eta) < 2.4) & ((isotrack.rel_iso < 0.1) | ((isotrack.rel_iso*isotrack.pt) < 6))]
        
        ## FatJets
        fatjet = JaggedCandidateArray.candidatesfromcounts(
            df['nFatJet'],
            pt = df['FatJet_pt'].content,
            eta = df['FatJet_eta'].content,
            phi = df['FatJet_phi'].content,
            mass = df['FatJet_mass'].content,
            msoftdrop = df["FatJet_msoftdrop"].content,  
            deepTagMD_HbbvsQCD = df['FatJet_deepTagMD_HbbvsQCD'].content, 
            deepTagMD_WvsQCD = df['FatJet_deepTagMD_WvsQCD'].content, 
            deepTag_WvsQCD = df['FatJet_deepTag_WvsQCD'].content
            
        )
        
        leadingFatJets = fatjet[:,:2]
        difatjet = leadingFatJets.choose(2)
        dphiDiFatJet = np.arccos(np.cos(difatjet.i0.phi-difatjet.i1.phi))
        
        min_dphiFatJetMet4 = np.arccos(np.cos(fatjet[:,:4].phi-met_phi)).min()
        
        htag = fatjet[((fatjet.pt > 200) & (fatjet.deepTagMD_HbbvsQCD > 0.8365))]
        htag_hard = fatjet[((fatjet.pt > 300) & (fatjet.deepTagMD_HbbvsQCD > 0.8365))]
        
        lead_htag = htag[htag.pt.argmax()]

        wtag = fatjet[((fatjet.pt > 200) & (fatjet.deepTagMD_HbbvsQCD < 0.8365) & (fatjet.deepTag_WvsQCD > 0.918))]
        wtag_hard = fatjet[((fatjet.pt > 300) & (fatjet.deepTagMD_HbbvsQCD < 0.8365) & (fatjet.deepTag_WvsQCD > 0.918))]
        
        lead_wtag = wtag[wtag.pt.argmax()]
        
        wh = lead_htag.cross(lead_wtag)
        wh_deltaPhi = np.arccos(wh.i0.phi - wh.i1.phi)
        wh_deltaR = wh.i0.p4.delta_r(wh.i1.p4)
        
        ## Jets
        jet = JaggedCandidateArray.candidatesfromcounts(
            df['nJet'],
            pt = df['Jet_pt'].content,
            eta = df['Jet_eta'].content,
            phi = df['Jet_phi'].content,
            mass = df['Jet_mass'].content,
            jetId = df['Jet_jetId'].content, # https://twiki.cern.ch/twiki/bin/view/CMS/JetID
            #puId = df['Jet_puId'].content, # https://twiki.cern.ch/twiki/bin/viewauth/CMS/PileupJetID
            btagDeepB = df['Jet_btagDeepB'].content, # https://twiki.cern.ch/twiki/bin/viewauth/CMS/BtagRecommendation102X
            #deepJet = df['Jet_'].content # not there yet?
        )
        
        skimjet   = jet[(jet.pt>30) & (abs(jet.eta)<2.4)]
        jet       = jet[(jet.pt>30) & (jet.jetId>1) & (abs(jet.eta)<2.4)]
        jet       = jet[~jet.match(muon, deltaRCut=0.4)] # remove jets that overlap with muons
        jet       = jet[~jet.match(electron, deltaRCut=0.4)] # remove jets that overlap with electrons
        jet       = jet[~jet.match(fatjet, deltaRCut=1.2)] # remove AK4 jets that overlap with AK8 jets
        jet       = jet[jet.pt.argsort(ascending=False)] # sort the jets
        btag      = jet[(jet.btagDeepB>0.4184)]
        light     = jet[(jet.btagDeepB<0.4184)]
        
        ## Get the leading b-jets
        high_score_btag = jet[jet.btagDeepB.argsort(ascending=False)][:,:2]
        
        leading_jet    = jet[jet.pt.argmax()]
        leading_b      = btag[btag.pt.argmax()]
        
        bb = high_score_btag.choose(2)
        bb_deltaPhi = np.arccos(np.cos(bb.i0.phi-bb.i1.phi))
        bb_deltaR = bb.i0.p4.delta_r(bb.i1.p4)
        
        mtb = mt(btag.pt, btag.phi, met_pt, met_phi)
        min_mtb = mtb.min()
        mth = mt(htag.pt, htag.phi, met_pt, met_phi)

                
        ## other variables
        ht = jet.pt.sum()
        
        min_dphiJetMet4 = np.arccos(np.cos(jet[:,:4].phi-met_phi)).min()
        
        leadingJets = jet[:,:2]
        dijet = leadingJets.choose(2)
        dphiDiJet = np.arccos(np.cos(dijet.i0.phi-dijet.i1.phi))
        
        #filters
        good_vertices = df["Flag_goodVertices"]
        tighthalo = df["Flag_globalSuperTightHalo2016Filter"]
        noise_filter = df["Flag_HBHENoiseFilter"]
        noise_isofilter = df["Flag_HBHENoiseIsoFilter"]
        ecal_deadcell = df["Flag_EcalDeadCellTriggerPrimitiveFilter"]
        bad_pfmuon = df["Flag_BadPFMuonFilter"]
        ee_badsc = df["Flag_eeBadScFilter"]
       
        #trigger
        hlt_pfmet_250 = df["HLT_PFMET250_HBHECleaned"]
        hlt_pfmet_300 = df["HLT_PFMET300_HBHECleaned"]
        hlt_pfmet1_200 = df["HLT_PFMETTypeOne200_HBHE_BeamHaloCleaned"]
        hlt_pfmet_mht = df["HLT_PFMET120_PFMHT120_IDTight_PFHT60"]
        hlt_pfmetNoMu_mhtNoMu = df["HLT_PFMETNoMu120_PFMHTNoMu120_IDTight_PFHT60"]
        
        met_fsel = (good_vertices == 1) & (tighthalo == 1) & (noise_filter == 1) & (noise_isofilter == 1) & (ecal_deadcell == 1) & (bad_pfmuon == 1) & (ee_badsc == 1) 
        met_tsel = (hlt_pfmet_250 == 1).any() | (hlt_pfmet_300 == 1).any() | (hlt_pfmet1_200 == 1).any() | (hlt_pfmet_mht == 1).any() | (hlt_pfmetNoMu_mhtNoMu == 1).any()
        
        ## define selections (maybe move to a different file at some point)
        
        output['totalEvents']['all'] += len(df['weight'])
        
        # Cutflow
        processes = ['mC750_l1', 'WJets', 'QCD', 'TTJets', 'ZNuNu', 'ST', 'ttW', 'ttZ', 'WW', 'WZ/ZZ']
        #processes = ['mC750_l1', 'LL', 'QCD', 'ZNuNu']
        weight = df['weight']
        lumi = 60
        '''cutflow = Cutflow(output, df, lumi, processes, weight=weight)
        
        cutflow.addRow( 'good_vertices',   (good_vertices==1) )
        cutflow.addRow( 'tighthalo',   (tighthalo==1) )
        cutflow.addRow( 'noise_filter',   (noise_filter==1) )
        cutflow.addRow( 'noise_isofilter',   (noise_isofilter==1) )
        cutflow.addRow( 'ecal_deadcell',   (ecal_deadcell==1) )
        cutflow.addRow( 'bad_pfmuon',   (bad_pfmuon==1) )
        cutflow.addRow( 'ee_badsc',   (ee_badsc==1) )

        cutflow.addRow( 'triggers',   (met_tsel) )

        cutflow.addRow( 'electron veto',   (electron.counts==0) )
        cutflow.addRow( 'muon veto',   (muon.counts==0) )
        cutflow.addRow( 'tau veto',   (tau.counts==0) )
        cutflow.addRow( 'isotrack veto',   (isotrack.counts==0) )
        cutflow.addRow( 'MET>250',     (met_pt>250) )
        cutflow.addRow( 'njet2',       (jet.counts>=2) )
        cutflow.addRow( 'jetveto',       (jet.counts<=5) )
        cutflow.addRow( 'nbtag',       (btag.counts>=1) )
        
        baseline = copy.deepcopy(cutflow.selection)
        
        cutflow.addRow( 'min_mt_met_b>200', (min_mtb>200) )
        cutflow.addRow( 'min_dphiJetMet4', (min_dphiJetMet4>0.5))
        cutflow.addRow( 'dphiDiJet', (dphiDiJet.min()<2.5) ) # the min doesn't do anything here
        cutflow.addRow( 'dphiDiFatJet', (dphiDiFatJet<2.5).all() ) # by using .all() I do not implicitely cut on the number of fat jets
        
        vetoQCD = copy.deepcopy(cutflow.selection)
        
        cutflow.addRow( 'HT>300',      (ht>300) )
        cutflow.addRow( 'N_fatjet>1',      (fatjet.counts>1) )
        cutflow.addRow( 'N_wtag>0',     (wtag.counts>0))
        cutflow.addRow( 'N_htag>0',     (htag.counts>0))
        #cutflow.addRow( 'H_mass>90',     ((lead_htag.msoftdrop>90).any()))
        #cutflow.addRow( 'H_mass<150',     ((lead_htag.msoftdrop<150).any()))
        #cutflow.addRow( 'N_fatjet>1',      (fatjet.counts>1) )
        
        event_selection = copy.deepcopy(cutflow.selection)
        
        #cutflow.addRow( 'N_htag>0 hard',     (htag_hard.counts>0))
        #cutflow.addRow( 'MET>400',     (met_pt>400) )
        #cutflow.addRow( 'MET>600',     (met_pt>600) )

        # signal enriched selection of events
        signal_selection = cutflow.selection'''
        
        
        cutflow = Cutflow(output, df, lumi, processes, weight=weight)
        
        cutflow.addRow( 'good_vertices',   (good_vertices==1) )
        cutflow.addRow( 'tighthalo',   (tighthalo==1) )
        cutflow.addRow( 'noise_filter',   (noise_filter==1) )
        cutflow.addRow( 'noise_isofilter',   (noise_isofilter==1) )
        cutflow.addRow( 'ecal_deadcell',   (ecal_deadcell==1) )
        cutflow.addRow( 'bad_pfmuon',   (bad_pfmuon==1) )
        cutflow.addRow( 'ee_badsc',   (ee_badsc==1) )

        cutflow.addRow( 'triggers',   (met_tsel) )
        
        cutflow.addRow( 'skim',   ((met_pt>200) & (skimjet.counts>1)) )
        cutflow.addRow( 'electron veto',   (electron.counts==0) )
        cutflow.addRow( 'muon veto',   (muon.counts==0) )
        cutflow.addRow( 'tau veto',    (tau.counts==0) )
        cutflow.addRow( 'iso track veto',   (isotrack.counts==0) )
        cutflow.addRow( 'MET>250',     (met_pt>250) )
        
        #cutflow.addRow( 'njet2',       (jet.counts>=2) )
        #cutflow.addRow( 'jetveto',     (jet.counts<=5) )
        #cutflow.addRow( 'nbtag',       (btag.counts>=1) )
        #cutflow.addRow( 'minmtb',      (mtb.min()>200) )
        
        baseline = copy.deepcopy(cutflow.selection)
        
        cutflow.addRow( 'N_fatjet>1',      (fatjet.counts>1) )
        #cutflow.addRow( 'min_dphiJetMet4', (min_dphiJetMet4>0.5))
        cutflow.addRow( 'min_dphiFatJetMet4', (min_dphiFatJetMet4>0.5))
        #cutflow.addRow( 'dphiDiJet', (dphiDiJet<2.5).all() ) # the min doesn't do anything here
        cutflow.addRow( 'dphiDiFatJet', (dphiDiFatJet<2.5).all() ) # by using .all() I do not implicitely cut on the number of fat jets
        
        vetoQCD = copy.deepcopy(cutflow.selection)
        
        #cutflow.addRow( 'HT>400',      (ht>400) )
        #cutflow.addRow( 'N_fatjet>0',      (fatjet.counts>0) )
        #cutflow.addRow( 'N_fatjet>1',      (fatjet.counts>1) )
        #cutflow.addRow( 'N_vlwtag>0',    (wtag_vloose.counts>0))
        #cutflow.addRow( 'N_lwtag>0',    (wtag_loose.counts>0))
        
        twotag_selection = copy.deepcopy(cutflow.selection)
        
        cutflow.addRow( 'minmth>200',   (mth.min()>200) )
        
        #cutflow.addRow( 'minmth>500',   (mth.min()>500) )
        #cutflow.addRow( 'minmth>600',   (mth.min()>600) )
        
        cutflow.addRow( 'njet veto',     (jet.counts<2))
        cutflow.addRow( 'N_htag>0',     (htag.counts>0))
        cutflow.addRow( 'on-H',     (abs(htag.msoftdrop-125)<25).any())
        cutflow.addRow( 'N_wtag>0',     (wtag.counts>0))
        cutflow.addRow( 'on-W',     (abs(wtag.msoftdrop-80)<30).any())

        event_selection = copy.deepcopy(cutflow.selection)
        
        #cutflow.addRow( 'N_htag>0 hard',     (htag_hard.counts>0))
        #cutflow.addRow( 'MET>400',     (met_pt>400) )
        #cutflow.addRow( 'no AK4',        (jet.counts==0))

        # signal enriched selection of events
        signal_selection = cutflow.selection
        
        ### And fill the histograms
        output['MET_pt_baseline'].fill(dataset=dataset, pt=met_pt[baseline].flatten(), weight=df['weight'][baseline]*cfg['lumi'])
        output['HT_baseline'].fill(dataset=dataset, ht=ht[baseline].flatten(), weight=df['weight'][baseline]*cfg['lumi'])
        output['mtb_min_baseline'].fill(dataset=dataset, mass=mtb[baseline].min().flatten(), weight=df['weight'][baseline]*cfg['lumi'])

        output['MET_pt'].fill(dataset=dataset, pt=met_pt[vetoQCD].flatten(), weight=df['weight'][vetoQCD]*cfg['lumi'])
        output['HT'].fill(dataset=dataset, ht=ht[vetoQCD].flatten(), weight=df['weight'][vetoQCD]*cfg['lumi'])
        output['mtb_min'].fill(dataset=dataset, mass=mtb[vetoQCD].min().flatten(), weight=df['weight'][vetoQCD]*cfg['lumi'])
        
        ## N jet and N b without selections on those
        output['N_AK4'].fill(dataset=dataset, multiplicity=jet[baseline].counts, weight=df['weight'][baseline]*cfg['lumi'])
        output['N_b'].fill(dataset=dataset, multiplicity=btag[baseline].counts, weight=df['weight'][baseline]*cfg['lumi'])       
        output['N_W'].fill(dataset=dataset, multiplicity=htag[baseline].counts, weight=df['weight'][baseline]*cfg['lumi'])       
        output['N_H'].fill(dataset=dataset, multiplicity=wtag[baseline].counts, weight=df['weight'][baseline]*cfg['lumi'])       
        output['N_AK8'].fill(dataset=dataset, multiplicity=fatjet[baseline].counts, weight=df['weight'][baseline]*cfg['lumi'])       

        #output['bb_deltaPhi'].fill(dataset=dataset, delta=bb_deltaPhi[baseline].flatten(), weight=df['weight'][baseline]*cfg['lumi'])
        #output['bb_deltaR'].fill(dataset=dataset, delta=bb_deltaR[baseline].flatten(), weight=df['weight'][baseline]*cfg['lumi'])

        output['min_dphiJetMet4'].fill(dataset=dataset, delta=min_dphiJetMet4[baseline].flatten(), weight=df['weight'][baseline]*cfg['lumi'])
        output['dphiDiJet'].fill(dataset=dataset, delta=dphiDiJet[baseline].min().flatten(), weight=df['weight'][baseline]*cfg['lumi'])

        ## Higgs and W pt
        output['lead_AK8_pt'].fill(dataset=dataset, pt=fatjet[(baseline & (fatjet.counts>0))].pt.max().flatten(), weight=df['weight'][(baseline & (fatjet.counts>0))]*cfg['lumi'])
        output['dphiDiFatJet'].fill(dataset=dataset, delta=dphiDiFatJet[(baseline & (fatjet.counts>1))].min().flatten(), weight=df['weight'][(baseline & (fatjet.counts>1))]*cfg['lumi'])

        output['H_pt'].fill(dataset=dataset, pt=lead_htag[event_selection].pt.flatten(), weight=df['weight'][event_selection]*cfg['lumi'])
        output['H_eta'].fill(dataset=dataset, eta=lead_htag[event_selection].eta.flatten(), weight=df['weight'][event_selection]*cfg['lumi'])

        output['W_pt'].fill(dataset=dataset, pt=lead_wtag[event_selection].pt.flatten(), weight=df['weight'][event_selection]*cfg['lumi'])
        output['W_eta'].fill(dataset=dataset, eta=lead_wtag[event_selection].eta.flatten(), weight=df['weight'][event_selection]*cfg['lumi'])

        output['WH_deltaPhi'].fill(dataset=dataset, delta=wh_deltaPhi[event_selection].flatten(), weight=df['weight'][event_selection]*cfg['lumi'])
        output['WH_deltaR'].fill(dataset=dataset, delta=wh_deltaR[event_selection].flatten(), weight=df['weight'][event_selection]*cfg['lumi'])

        output['MET_pt_SR'].fill(dataset=dataset, pt=met_pt[event_selection].flatten(), weight=df['weight'][event_selection]*cfg['lumi'])
        output['HT_SR'].fill(dataset=dataset, ht=ht[event_selection].flatten(), weight=df['weight'][event_selection]*cfg['lumi'])
        output['mtb_min_SR'].fill(dataset=dataset, mass=mtb[event_selection].min().flatten(), weight=df['weight'][event_selection]*cfg['lumi'])
        
        return output

    def postprocess(self, accumulator):
        return accumulator

In [11]:
runLocal = True


if not runLocal:
    # Get the scheduler from the dask_cluster notebook
    from dask.distributed import Client, progress

    c = Client('tcp://169.228.130.5:27879')

    ## for dask
    exe_args = {
        'client': c,
        #'savemetrics': True,
    }
    exe = processor.dask_executor
    
else:
    ## for local
    exe_args = {
        'workers': 4,
        'function_args': {'flatten': False}
    }
    exe = processor.futures_executor

if not runLocal:
    print(c)

In [12]:
overwrite = True
small = False

tag = '0p1p27'
#tag = '0p1p31/2018'
#tag = '0p1p16/2018'

fileset_WH   = {'mC750_l1': glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/WH_had_750_1_nanoAOD/*.root'),
                'WJets': glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/W*JetsToLNu_Tune*/*.root'),
                'QCD': glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/QCD_HT*/*.root'),
                'TTJets': glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/TTJets*/*.root'),
                'ZNuNu': glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/ZJetsToNuNu*/*.root'),
                'ST': glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/ST*/*.root'),
                'ttW': glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/ttWJets*/*.root'),
                'ttZ': glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/ttZJets*/*.root'),
                'WW': glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/WW*/*.root'),
                'WZ/ZZ': glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/WZ*/*.root')
                    +glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/ZZTo2L2Nu*/*.root')
                    +glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/ZZTo2Q2Nu*/*.root')
                }

fileset_WH_merge = {'mC750_l1': glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/WH_had_750_1_nanoAOD/*.root'),
                'LL': glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/WJetsToLNu*/*.root')
                    + glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/TTJets*/*.root')
                    + glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/ST*/*.root')
                    + glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/ttWJets*/*.root')
                    + glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/WW*/*.root'),
                'QCD': glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/QCD_HT*/*.root'),
                'ZNuNu': glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/ZJetsToNuNu*/*.root')
                    + glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/ttZJets*/*.root')
                    + glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/WZ*/*.root')
                    + glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/ZZTo2L2Nu*/*.root')
                    + glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/ZZTo2Q2Nu*/*.root')
                }


# load the config and the cache
cfg = loadConfig()

cacheName = 'WH_small' if small else 'WH'

# histograms
histograms = []
histograms += ['N_AK4']

# initialize cache
cache = dir_archive(os.path.join(os.path.expandvars(cfg['caches']['base']), cfg['caches'][cacheName]), serialized=True)
if not overwrite:
    cache.load()

if cfg == cache.get('cfg') and histograms == cache.get('histograms') and cache.get('simple_output'):
    output = cache.get('simple_output')

else:
    # Run the processor
    if small:
        fileset = {
                    'mC750_l1': glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/WH_had_750_1_nanoAOD/*.root'),
                    'LL': glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/ttWJets*/*.root')[:2],
                    'QCD': glob.glob('/hadoop/cms/store/user/ksalyer/allHadTest/'+tag+'/QCD_HT*/*.root')[:2]}
        workers = 4
    else:
        fileset = fileset_WH
        workers = 16
    
        
    output = processor.run_uproot_job(fileset,
                                      treename='Events',
                                      processor_instance=analysisProcessor(),
                                      executor=exe,
                                      executor_args=exe_args,
                                      #chunksize=250000,
                                      chunksize=100000,
                                     )
    cache['fileset']        = fileset
    cache['cfg']            = cfg
    cache['histograms']     = histograms
    cache['simple_output']  = output
    cache.dump()

HBox(children=(HTML(value='Processing'), FloatProgress(value=0.0, max=400.0), HTML(value='')))

/home/users/ksalyer/CMSSW_10_2_9/src/tW_scattering/coffeaEnv/lib/python3.6/site-packages/awkward/array/jagged.py:1043: RuntimeWarning: invalid value encountered in arccos
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/users/ksalyer/CMSSW_10_2_9/src/tW_scattering/coffeaEnv/lib/python3.6/site-packages/awkward/array/jagged.py:1043: RuntimeWarning: invalid value encountered in arccos
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/users/ksalyer/CMSSW_10_2_9/src/tW_scattering/coffeaEnv/lib/python3.6/site-packages/awkward/array/jagged.py:1043: RuntimeWarning: invalid value encountered in arccos
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/users/ksalyer/CMSSW_10_2_9/src/tW_scattering/coffeaEnv/lib/python3.6/site-packages/awkward/array/jagged.py:1043: RuntimeWarning: invalid value encountered in arccos
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [13]:
# Cutflow
from Tools.helpers import getCutFlowTable

processes = ['mC750_l1', 'WJets', 'QCD', 'TTJets', 'ZNuNu', 'ST', 'ttW', 'ttZ', 'WW', 'WZ/ZZ']
#processes = ['mC750_l1', 'LL', 'QCD', 'ZNuNu']
lines     = ['entry']
#lines    += [ 'electron veto', 'muon veto', 'tau veto', 'isotrack veto', 'MET>250','njet2', 'jetveto', 'nbtag', 'min_mt_met_b>200', 'min_dphiJetMet4', 'dphiDiJet', 'dphiDiFatJet',  'HT>300', 'N_fatjet>1', 'N_wtag>0', 'N_htag>0']
#lines    += ['skim', 'electron veto', 'muon veto', 'tau veto', 'iso track veto',  'MET>250', 'N_fatjet>1', 'min_dphiFatJetMet4', 'dphiDiFatJet', 'N_htag>0', 'N_wtag>0', 'minmth>200', 'on-H', 'on-W', 'njet veto',  'MET>400', 'no AK4']
lines    += ['skim', 'electron veto', 'muon veto', 'tau veto', 'iso track veto',  'MET>250', 'N_fatjet>1', 'min_dphiFatJetMet4', 'dphiDiFatJet', 'minmth>200', 'njet veto', 'N_htag>0', 'on-H', 'N_wtag>0', 'on-W']
#lines     = ['entry']
#lines     = ['skim', 'electron veto', 'muon veto', 'tau veto', 'iso track veto', 'MET>250', 'N_fatjet>1', 'min_dphiFatJetMet4', 'dphiDiFatJet', 'N_htag>0', 'N_wtag>0', 'minmth>200', 'on-H', 'on-W', 'njet veto', 'MET>400', 'no AK4']
df        = getCutFlowTable(output, processes=processes, lines=lines, significantFigures=4, signal='mC750_l1')
df

,mC750_l1,WJets,QCD,TTJets,ZNuNu,ST,ttW,ttZ,WW,WZ/ZZ,S/B
entry,130.5 +/- 0.6,1715000.0 +/- 3000.0,1712000.0 +/- 12000.0,682200.0 +/- 400.0,942600.0 +/- 500.0,76370.0 +/- 100.0,1939.0 +/- 2.0,3276.0 +/- 3.0,27020.0 +/- 160.0,22380.0 +/- 80.0,0.0000
skim,129.7 +/- 0.6,1611000.0 +/- 3000.0,1107000.0 +/- 8000.0,676800.0 +/- 400.0,862500.0 +/- 500.0,74040.0 +/- 100.0,1927.0 +/- 2.0,3261.0 +/- 3.0,25820.0 +/- 150.0,21150.0 +/- 80.0,0.0000
electron veto,128.9 +/- 0.6,1240000.0 +/- 3000.0,1102000.0 +/- 8000.0,495000.0 +/- 300.0,859000.0 +/- 500.0,54390.0 +/- 90.0,1286.0 +/- 2.0,2507.0 +/- 2.0,18720.0 +/- 130.0,17680.0 +/- 70.0,0.0000
muon veto,127.6 +/- 0.6,803200.0 +/- 2200.0,1012000.0 +/- 6000.0,297800.0 +/- 200.0,855500.0 +/- 500.0,33150.0 +/- 70.0,681.7 +/- 1.4,1740.0 +/- 2.0,11250.0 +/- 100.0,13920.0 +/- 60.0,0.0000
tau veto,125.8 +/- 0.6,643300.0 +/- 1900.0,998500.0 +/- 6000.0,242300.0 +/- 200.0,845800.0 +/- 500.0,26100.0 +/- 60.0,537.8 +/- 1.2,1527.0 +/- 2.0,8698.0 +/- 89.0,12620.0 +/- 60.0,0.0000
iso track veto,121.6 +/- 0.6,597700.0 +/- 1800.0,965100.0 +/- 5900.0,217400.0 +/- 200.0,821600.0 +/- 500.0,23690.0 +/- 60.0,476.6 +/- 1.2,1393.0 +/- 2.0,8013.0 +/- 85.0,12030.0 +/- 60.0,0.0000
MET>250,111.0 +/- 0.6,230100.0 +/- 1100.0,248000.0 +/- 2100.0,81350.0 +/- 120.0,382900.0 +/- 300.0,8649.0 +/- 35.0,231.4 +/- 0.8,760.6 +/- 1.3,3318.0 +/- 55.0,5881.0 +/- 38.0,0.0001
N_fatjet>1,72.94 +/- 0.46,45030.0 +/- 440.0,181800.0 +/- 1000.0,28540.0 +/- 70.0,67280.0 +/- 70.0,2651.0 +/- 19.0,125.6 +/- 0.6,287.8 +/- 0.8,515.1 +/- 21.6,852.0 +/- 14.3,0.0002
min_dphiFatJetMet4,68.26 +/- 0.45,30980.0 +/- 370.0,9582.0 +/- 564.0,12160.0 +/- 50.0,55350.0 +/- 60.0,1248.0 +/- 13.0,58.99 +/- 0.41,199.9 +/- 0.6,295.7 +/- 16.4,695.2 +/- 12.7,0.0006
dphiDiFatJet,55.99 +/- 0.41,23310.0 +/- 320.0,7315.0 +/- 233.0,8181.0 +/- 39.0,42280.0 +/- 60.0,893.8 +/- 11.4,38.17 +/- 0.33,146.3 +/- 0.6,237.6 +/- 14.7,573.3 +/- 11.7,0.0007


In [14]:

# Efficiencies
df = getCutFlowTable(output, processes=processes, lines=lines, significantFigures=3, absolute=False)
df

,mC750_l1,WJets,QCD,TTJets,ZNuNu,ST,ttW,ttZ,WW,WZ/ZZ
entry,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000
skim,0.994,0.939,0.647,0.992,0.915,0.969,0.994,0.996,0.955,0.945
electron veto,0.994,0.770,0.995,0.731,0.996,0.735,0.668,0.769,0.725,0.836
muon veto,0.990,0.648,0.919,0.602,0.996,0.609,0.530,0.694,0.601,0.788
tau veto,0.986,0.801,0.986,0.814,0.989,0.787,0.789,0.877,0.773,0.907
iso track veto,0.967,0.929,0.967,0.897,0.971,0.907,0.886,0.912,0.921,0.953
MET>250,0.913,0.385,0.257,0.374,0.466,0.365,0.486,0.546,0.414,0.489
N_fatjet>1,0.657,0.196,0.733,0.351,0.176,0.307,0.543,0.378,0.155,0.145
min_dphiFatJetMet4,0.936,0.688,0.053,0.426,0.823,0.471,0.470,0.695,0.574,0.816
dphiDiFatJet,0.820,0.752,0.763,0.673,0.764,0.716,0.647,0.732,0.804,0.825
